<center><h1 style="text-align: center;"><b>Best Hack 2023</b></h1><center>
<center><h2 style="text-align: center;"><b>GibData Solution</b></h2><center>
    

<h3 style="text-align: center;"><b>Задача: Разработать модель машинного обучения, способную прогнозировать остаточный ресурс колёсной пары</b></h3>

---
## Библиотеки

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import scipy
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, ShuffleSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
from catboost import Pool, cv, CatBoostRegressor

%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

## Загрузка данных

In [2]:
train_df = pd.read_parquet("Данные для полуфинала Задача №1/train.parquet")
train_target = pd.read_csv("Данные для полуфинала Задача №1/train_target.csv")

test_df = pd.read_parquet("Данные для полуфинала Задача №1/test.parquet")
test_submit_example = pd.read_csv("Данные для полуфинала Задача №1/test_submit_example.csv")

In [3]:
train_df.shape, train_target.shape, test_df.shape, test_submit_example.shape

((627228, 11), (33012, 3), (209076, 11), (11004, 3))

In [4]:
train_df.head(19)

,wagnum,ts_id,milleage_all,axl1_l_w_flange,axl1_r_w_flange,axl2_l_w_flange,axl2_r_w_flange,axl3_l_w_flange,axl3_r_w_flange,axl4_l_w_flange,axl4_r_w_flange
0,2,3,10371.0,28.6,29.7,28.6,28.8,30.4,29.3,29.9,28.3
1,2,3,29448.0,28.8,29.1,28.0,27.8,30.1,28.9,29.1,27.8
2,2,3,33963.0,28.6,29.1,28.3,28.5,29.8,29.4,29.7,28.3
3,2,3,37184.0,29.0,29.7,29.1,29.0,30.0,30.2,29.0,28.0
4,2,3,37637.0,29.0,29.0,28.5,29.4,30.0,29.9,28.9,28.7
5,2,3,41181.0,29.3,29.9,28.8,28.2,30.2,29.8,30.8,27.5
6,2,3,41960.0,28.9,29.3,28.5,28.6,29.7,28.8,30.1,28.2
7,2,3,42340.0,29.1,28.9,28.7,28.7,30.0,29.9,29.7,27.3
8,2,3,43126.0,28.9,29.4,28.1,28.8,29.6,29.6,28.5,28.5
9,2,3,46747.0,29.0,28.8,28.6,28.4,29.9,29.7,29.1,27.8


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627228 entries, 0 to 627227
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   wagnum           627228 non-null  int64  
 1   ts_id            627228 non-null  int64  
 2   milleage_all     622599 non-null  float64
 3   axl1_l_w_flange  626871 non-null  float64
 4   axl1_r_w_flange  626899 non-null  float64
 5   axl2_l_w_flange  626853 non-null  float64
 6   axl2_r_w_flange  626881 non-null  float64
 7   axl3_l_w_flange  626837 non-null  float64
 8   axl3_r_w_flange  626892 non-null  float64
 9   axl4_l_w_flange  626819 non-null  float64
 10  axl4_r_w_flange  626916 non-null  float64
dtypes: float64(9), int64(2)
memory usage: 52.6 MB


## Предобработка

In [6]:
print("Количество уникальных вагонов в трейне:", train_df['wagnum'].nunique(),
      "\nКоличество уникальных вагонов в тесте: ", test_df['wagnum'].nunique(),
      "\n\nКоличество уникальных временных рядов в трейне:", train_df['ts_id'].nunique(),
      "\nКоличество уникальных временных рядов в тесте: ", test_df['ts_id'].nunique())

Количество уникальных вагонов в трейне: 29221 
Количество уникальных вагонов в тесте:  10601 

Количество уникальных временных рядов в трейне: 11 
Количество уникальных временных рядов в тесте:  11


In [7]:
# Распределения остаточного ресурса в зависимости от id времененного ряда
info_targets = pd.DataFrame({"value_counts": train_df['ts_id'].value_counts(),
                             "mean_target": train_target.groupby('ts_id').aggregate({'target': 'mean'})['target'],
                             "min_target": train_target.groupby('ts_id').aggregate({'target': 'min'})['target'],
                             "max_target": train_target.groupby('ts_id').aggregate({'target': 'max'})['target']})
info_targets

,value_counts,mean_target,min_target,max_target
1,118047,61206.816836,1159.0,301983.0
2,167257,45902.515165,611.0,213365.0
3,156427,43620.550103,890.0,185229.0
4,102543,37744.892162,872.0,206610.0
5,51414,30419.328160,680.0,160540.0
6,20824,26469.704380,1293.0,159051.0
7,7505,22587.037975,984.0,102975.0
8,2299,22000.090909,1900.0,113299.0
9,665,15741.800000,2789.0,62361.0
10,209,11765.090909,1612.0,26817.0


### Функия для предобработки данных

In [8]:
def prepare_data(data):
    '''
    Функция для подготовки данных к обучению или тестированию
    
    Аргументы:
    data -- pandas dataframe с исходными признаками
    
    Возвращает:
    pandas dataframe с извлеченными признаками для MetaModel и AutoML
    '''
    # вектор id временных рядов
    ts_id = [data['ts_id'][i] for i in range(0, data.shape[0], 19)]
    
    # маска выбросов
    sub_vec = data.milleage_all > data.milleage_all.quantile(0.75) + 1.5 * scipy.stats.iqr(data.milleage_all, nan_policy="omit")
    data['is_blowout'] = sub_vec.astype(int)
    
    # функции агрегации
    statistical_signs = ['last', 'min']
    features_cb = {}
    features_automl = {}
    
    # названия исходных столбцов
    cols = ['milleage_all', 'axl1_l_w_flange', 'axl1_r_w_flange', 'axl2_l_w_flange', 
            'axl2_r_w_flange', 'axl3_l_w_flange', 'axl3_r_w_flange', 'axl4_l_w_flange', 'axl4_r_w_flange', 'is_blowout']
    
    # создание словаря для агрегации
    for col in cols:
        if col == "is_blowout":
            features_cb[col] = 'last'
            features_automl[col] = 'last'
        else:
            features_cb[col] = statistical_signs
            features_automl[col] = statistical_signs + ['first', 'mean']
            
    # группировка и агрегация
    data_cb = data.groupby(['wagnum', 'ts_id']).aggregate(features_cb).reset_index()
    data_automl = data.groupby(['wagnum', 'ts_id']).aggregate(features_automl).reset_index()
    
    # создание итоговых фреймов с переименованием столбцов
    result_cb = pd.DataFrame()
    result_automl = pd.DataFrame()
    
    for col in cols[:-1]:
        for col_ in data_cb[col].columns:
            result_cb[col + "_" + col_] = data_cb[col][col_]
        for col_ in data_automl[col].columns:
            result_automl[col + "_" + col_] = data_automl[col][col_]
            
    result_cb['is_blowout'] = data_cb['is_blowout'].astype('category')
    result_automl['is_blowout'] = data_automl['is_blowout'].astype('category')
    
    for col in cols[1:-1]:
        result_automl[col + "_fi-last"] = result_automl[col + "_first"] - result_automl[col + "_last"]
    
    delete_cols = [col for col in result_automl.columns if "first" in col]
    result_automl = result_automl.drop(columns=delete_cols[1:])
    
    result_cb['ts_id'] = np.array(ts_id)
    result_cb['ts_id'] = result_cb['ts_id'].astype('category')
    result_automl['ts_id'] = result_cb['ts_id']
    
    return result_cb, result_automl

In [9]:
%%time
data_train_cb, data_train_automl = prepare_data(train_df)
data_test_cb, data_test_automl = prepare_data(test_df)

CPU times: total: 969 ms
Wall time: 984 ms


## Обучение моделей и проверка на отложенной выборке

### *CatBoost Regressor / Holdout Sample*

In [10]:
X_cb = data_train_cb
y = train_target['target']

X_train, X_holdout, y_train, y_holdout = train_test_split(X_cb, y, test_size=0.33, random_state=17, shuffle=True)
print(X_train.shape, X_holdout.shape)

(22118, 20) (10894, 20)


In [11]:
cat_cols = ['ts_id', 'is_blowout']

In [12]:
model = CatBoostRegressor(
    random_seed=63,
    loss_function='MAE',
    iterations=3000,
    learning_rate=0.05,
    task_type='CPU',
)

model.fit(
    Pool(data=X_train, label=y_train, cat_features=cat_cols),
    verbose=50,
    eval_set=Pool(data=X_holdout, label=y_holdout, cat_features=cat_cols),
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 26460.3553247	test: 26941.0392202	best: 26941.0392202 (0)	total: 79.2ms	remaining: 3m 57s
50:	learn: 18596.6489191	test: 19046.0369743	best: 19046.0369743 (50)	total: 1.74s	remaining: 1m 40s
100:	learn: 17662.7966253	test: 18189.4193553	best: 18189.4193553 (100)	total: 3.28s	remaining: 1m 34s
150:	learn: 17173.4711957	test: 17795.7239245	best: 17795.7239245 (150)	total: 4.73s	remaining: 1m 29s
200:	learn: 16861.4621643	test: 17572.6548620	best: 17572.6548620 (200)	total: 6.2s	remaining: 1m 26s
250:	learn: 16632.4919843	test: 17440.4077525	best: 17440.4077525 (250)	total: 7.62s	remaining: 1m 23s
300:	learn: 16425.2336249	test: 17322.5580099	best: 17321.3712457 (299)	total: 9.04s	remaining: 1m 21s
350:	learn: 16248.1599480	test: 17244.3246128	best: 17244.3246128 (350)	total: 10.6s	remaining: 1m 20s
400:	learn: 16097.7137059	test: 17185.5981266	best: 17185.5981266 (400)	total: 12.1s	remaining: 1m 18s
450:	learn: 15968.5394089	test: 17144.0621461	best: 17144.0621461 (450)	total: 

In [13]:
pred_catboost = model.predict(Pool(X_holdout, cat_features=cat_cols))
print("Testing Performance:", # 16943.91418275833   ->  statistical_signs = ['last', 'min'] + is_blowout
      "\nMAE:", mean_absolute_error(y_holdout, pred_catboost),
      "\nMSE:", mean_squared_error(y_holdout, pred_catboost))

Testing Performance: 
MAE: 16943.91418275833 
MSE: 598932675.5497494


In [14]:
fi = model.get_feature_importance(Pool(X_train, y_train, cat_features=cat_cols), prettified=True)
fi.head(19)

,Feature Id,Importances
0,milleage_all_min,10.326031
1,ts_id,10.061709
2,axl2_l_w_flange_min,7.217976
3,milleage_all_last,6.198130
4,axl3_r_w_flange_min,5.451088
5,axl1_r_w_flange_min,5.278540
6,axl3_r_w_flange_last,5.043659
7,axl4_r_w_flange_min,4.975931
8,axl3_l_w_flange_last,4.829226
9,axl1_l_w_flange_last,4.788607


In [16]:
params = {
    'loss_function': 'MAE',
    'iterations': 1500,
    'learning_rate': 0.05,
    'task_type':'CPU',
}

shuffle_split = ShuffleSplit(n_splits=5, test_size=.33, random_state=0)
cv_data = cv(
    params=params,
    pool=Pool(data=X_cb, label=y, cat_features=cat_cols),
    verbose=False,
    folds=shuffle_split,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]

bestTest = 16801.45807
bestIteration = 1390

Training on fold [1/5]

bestTest = 16868.99421
bestIteration = 1456

Training on fold [2/5]

bestTest = 16765.30844
bestIteration = 1477

Training on fold [3/5]

bestTest = 17073.43511
bestIteration = 1498

Training on fold [4/5]

bestTest = 16771.88318
bestIteration = 1499



### *MetaModel*

In [17]:
from sklearn.model_selection import StratifiedKFold
import copy

class MetaModel:
    def __init__(self, params):
        self.models = []
        self.params = params
        self.scores = []
        
    def fit(self, **kwargs):
        self.models = []
        self.scores = []
        
        for i in range(5):
            print(f"Model #{i+1}")
            now_params = self.params.copy()
            now_params['random_state'] = 42 + i
            now_model = CatBoostRegressor(**now_params)
            now_model.fit(**kwargs)

            preds = now_model.predict(X_holdout)
            mae = mean_absolute_error(y_holdout, preds)
            print(f'Random state {i + 42} mae = {mae}')
            
            self.scores.append(mae)
            self.models.append(now_model)
            now_model.save_model(f'models_weights/catboost_model_#{i+1}.bin')
            print(f"The model #{i+1} is saved")
            print('---------------------------------------------')
        print('---------------------------------------------')
        print('\nMean acc = {}'.format(np.mean(self.scores)))
    
    def predict(self, X):
        if len(self.models) == 0:
            raise Exception("There is not fitted model")
            
        preds = np.concatenate([model.predict(X).reshape(-1, 1) for model in self.models], axis=1)
        return preds.mean(axis=1)

In [18]:
params = {
    'n_estimators' : 3000,
    'learning_rate': 0.05,
    'cat_features' : cat_cols,
    'use_best_model': False,
    'task_type': 'CPU',
    'random_state': 42,
    'loss_function': 'MAE',
}

In [19]:
meta_model = MetaModel(params)
meta_model.fit(X=X_train, y=y_train, verbose=500, eval_set=(X_holdout, y_holdout), early_stopping_rounds=500)

Model #1
0:	learn: 26469.9848844	test: 26942.7539715	best: 26942.7539715 (0)	total: 26ms	remaining: 1m 18s
500:	learn: 15855.1155801	test: 17135.8908269	best: 17135.5626309 (499)	total: 15s	remaining: 1m 14s
1000:	learn: 15103.2429775	test: 17032.4659376	best: 17030.8557151 (965)	total: 29.6s	remaining: 59s
1500:	learn: 14605.6185001	test: 17004.4454504	best: 17004.4454504 (1500)	total: 44.6s	remaining: 44.6s
2000:	learn: 14257.4789139	test: 17006.3370365	best: 17003.4322609 (1972)	total: 59.4s	remaining: 29.6s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 17003.43226
bestIteration = 1972

Random state 42 mae = 17004.619115228496
The model #1 is saved
---------------------------------------------
Model #2
0:	learn: 26459.2451090	test: 26928.8353004	best: 26928.8353004 (0)	total: 36.6ms	remaining: 1m 49s
500:	learn: 15873.1194030	test: 17125.2995334	best: 17125.2599299 (498)	total: 15.4s	remaining: 1m 16s
1000:	learn: 15074.8085981	test: 16998.8911488	best: 16998.89

In [20]:
pred_metamodel = meta_model.predict(X_holdout)
mean_absolute_error(y_holdout, pred_metamodel)

16902.284719118135

### *AutoML*

In [22]:
X_automl = data_train_automl
y = train_target['target']

X_train, X_holdout, y_train, y_holdout = train_test_split(X_automl, y, test_size=0.33, random_state=17, shuffle=True)
print(X_train.shape, X_holdout.shape)

(22118, 38) (10894, 38)


In [23]:
from flaml import AutoML

automl = AutoML()
automl.fit(X_train,
           y_train,
           task="regression",
           metric='mae',
           time_budget=1000)

[flaml.automl.automl: 04-27 19:40:50] {2715} INFO - task = regression
[flaml.automl.automl: 04-27 19:40:50] {2717} INFO - Data split method: uniform
[flaml.automl.automl: 04-27 19:40:50] {2720} INFO - Evaluation method: cv
[flaml.automl.automl: 04-27 19:40:50] {2847} INFO - Minimizing error metric: mae
[flaml.automl.automl: 04-27 19:40:50] {2993} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl.automl: 04-27 19:40:50] {3322} INFO - iteration 0, current learner lgbm
[flaml.automl.automl: 04-27 19:40:50] {3460} INFO - Estimated sufficient time budget=4614s. Estimated necessary time budget=39s.
[flaml.automl.automl: 04-27 19:40:50] {3507} INFO -  at 0.8s,	estimator lgbm's best error=26058.3346,	best estimator lgbm's best error=26058.3346
[flaml.automl.automl: 04-27 19:40:50] {3322} INFO - iteration 1, current learner lgbm
[flaml.automl.automl: 04-27 19:40:51] {3507} INFO -  at 1.2s,	estimator lgbm's best error=26

In [24]:
pred_automl = automl.predict(X_holdout)

In [25]:
print("MAE:", mean_absolute_error(y_holdout, pred_automl), # -> 16662.732019549447 
      "\nMSE:", mean_squared_error(y_holdout, pred_automl),)

MAE: 16731.807395390202 
MSE: 559115081.5479803


### *Блендинг AutoML и CatBoost*

In [26]:
pred_cv_automl = (pred_automl + pred_metamodel) / 2
print("MAE:", mean_absolute_error(y_holdout, pred_cv_automl),
      "\nMSE:", mean_squared_error(y_holdout, pred_cv_automl))

MAE: 16651.789749102252 
MSE: 568033014.9432174


## Финальное обучение моделей

In [27]:
class MetaModelFinal(MetaModel):
    def fit(self, **kwargs):
        self.models = []
        self.scores = []
        
        for i in range(5):
            print(f"Model #{i+1}")
            now_params = self.params.copy()
            now_params['random_state'] = 42 + i
            now_model = CatBoostRegressor(**now_params)
            now_model.fit(**kwargs)
            
            self.models.append(now_model)
            now_model.save_model(f'models_weights/catboost_model_#{i+1}.bin')
            print(f"The model #{i+1} is saved")
            print('---------------------------------------------')
        print('---------------------------------------------')
        print('\nModels are trained on all data')

In [28]:
meta_model = MetaModelFinal(params)
meta_model.fit(X=X_cb, y=y, verbose=500, early_stopping_rounds=500)

Model #1
0:	learn: 26614.0020700	total: 30.6ms	remaining: 1m 31s
500:	learn: 16120.7942883	total: 16.7s	remaining: 1m 23s
1000:	learn: 15465.5929941	total: 34.3s	remaining: 1m 8s
1500:	learn: 15060.3210851	total: 51.1s	remaining: 51s
2000:	learn: 14787.0024933	total: 1m 6s	remaining: 33.4s
2500:	learn: 14581.6489893	total: 1m 22s	remaining: 16.5s
2999:	learn: 14412.8404075	total: 1m 38s	remaining: 0us
The model #1 is saved
---------------------------------------------
Model #2
0:	learn: 26604.3084009	total: 31.2ms	remaining: 1m 33s
500:	learn: 16157.8314053	total: 16.7s	remaining: 1m 23s
1000:	learn: 15517.0479508	total: 33.5s	remaining: 1m 6s
1500:	learn: 15115.2916063	total: 49.9s	remaining: 49.9s
2000:	learn: 14830.5092906	total: 1m 6s	remaining: 33.2s
2500:	learn: 14607.7225931	total: 1m 23s	remaining: 16.7s
2999:	learn: 14444.8793134	total: 1m 41s	remaining: 0us
The model #2 is saved
---------------------------------------------
Model #3
0:	learn: 26620.2910559	total: 29ms	remaini

In [29]:
automl = AutoML()
automl.fit(X_automl,
           y,
           task="regression",
           metric='mae',
           time_budget=2000)

[flaml.automl.automl: 04-27 20:09:49] {2715} INFO - task = regression
[flaml.automl.automl: 04-27 20:09:49] {2717} INFO - Data split method: uniform
[flaml.automl.automl: 04-27 20:09:49] {2720} INFO - Evaluation method: cv
[flaml.automl.automl: 04-27 20:09:49] {2847} INFO - Minimizing error metric: mae
[flaml.automl.automl: 04-27 20:09:49] {2993} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'catboost', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl.automl: 04-27 20:09:49] {3322} INFO - iteration 0, current learner lgbm
[flaml.automl.automl: 04-27 20:09:49] {3460} INFO - Estimated sufficient time budget=5455s. Estimated necessary time budget=47s.
[flaml.automl.automl: 04-27 20:09:49] {3507} INFO -  at 1.1s,	estimator lgbm's best error=26207.9240,	best estimator lgbm's best error=26207.9240
[flaml.automl.automl: 04-27 20:09:49] {3322} INFO - iteration 1, current learner lgbm
[flaml.automl.automl: 04-27 20:09:50] {3507} INFO -  at 1.7s,	estimator lgbm's best error=26

In [30]:
import pickle
with open("models_weights/automl.pkl", "wb") as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

### *Функция для предсказывания на данных*

In [31]:
import pickle


def make_submit(data):
    '''
    Функция для прогнозирования остаточного ресурса колёсной пары на заданной выборке
    
    Аргументы:
    data -- pandas dataframe с исходными признаками
    
    Возвращает:
    pandas dataframe с прогнозом
    '''
    # вектор id временных рядов
    ts_id = [data['ts_id'][i] for i in range(0, data.shape[0], 19)]
    
    # вектор id вагонов
    wagnums = [data['wagnum'][i] for i in range(0, data.shape[0], 19)]
    
    # подготовка данных
    data_cb, data_automl = prepare_data(data)
    
    # загрузка модели AutoML
    with open("models_weights/automl.pkl", "rb") as f:
        automl = pickle.load(f)
    
    # предсказывание модели AutoML и MetaModel
    pred_automl = automl.predict(data_automl)
    pred_metamodel = np.zeros(pred_automl.shape[0])
    for i in range(0, 5):
        model = CatBoostRegressor()
        model.load_model(f'models_weights/catboost_model_#{i+1}.bin')
        pred_metamodel = pred_metamodel + model.predict(data_cb)
    
    pred_metamodel = pred_metamodel / 5
    
    # финальный сабмит
    predictions = pd.DataFrame({'wagnum': wagnums,
                                'ts_id': ts_id,
                                'target': (pred_automl + pred_metamodel) / 2})
    return predictions

### *Финальный сабмит*

In [40]:
GibData = make_submit(test_df)
GibData.to_csv("GibData.csv")